# mnist数据集 on cnn

In [31]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

np.set_printoptions(suppress=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## CNN 模型

https://blog.csdn.net/weixin_42451919/article/details/81381294

In [32]:
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

In [33]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [34]:
#测试权重和偏值项初始化函数
with tf.Session() as sess:
    w=weight_variable([2,2])
    b=bias_variable([2,2])
    sess.run(tf.initialize_all_variables())
    print(sess.run(w))
    print(sess.run(b))

[[-0.10217106 -0.07968484]
 [ 0.07633816  0.00067627]]
[[0.1 0.1]
 [0.1 0.1]]


In [35]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')



def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


https://blog.csdn.net/zuolixiangfisher/article/details/80528989  参数解释   
https://blog.csdn.net/m0_37586991/article/details/84575325 参数解释

第一层卷积

In [36]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [37]:
x_image = tf.reshape(x, [-1,28,28,1])

In [38]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) #28*28*32  这一步 是广播运算
h_pool1 = max_pool_2x2(h_conv1)  # 输出应该是 14*14* 32 

第二层卷积

In [39]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)   # 7*7*64

全连接

In [40]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

dropout

In [41]:
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

输出层

In [42]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

训练与模型评估

In [43]:
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,log_device_placement=True))
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.initialize_all_variables())
for i in range(1000):
    batch = mnist.train.next_batch(50)
    sess.run(train_step,feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    if i%100 == 0:
        train_accuracy = sess.run(accuracy,feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
        




step 0, training accuracy 0.2
step 100, training accuracy 0.86
step 200, training accuracy 0.9
step 300, training accuracy 0.96
step 400, training accuracy 0.9
step 500, training accuracy 0.92
step 600, training accuracy 1
step 700, training accuracy 0.94
step 800, training accuracy 0.92
step 900, training accuracy 0.96


In [44]:
# 卡内存
#print("test accuracy %g"%sess.run(accuracy,feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

## 进行预测

In [45]:
imginx=10
img1 = mnist.train.images[imginx].reshape(-1,784)
print(img1.shape)
label1=mnist.train.labels[imginx].reshape([-1,10])
print(label1)


(1, 784)
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]


In [46]:
print(sess.run(y_conv, feed_dict={x: img1, y_: label1 , keep_prob: 1}))
print(sess.run(tf.argmax(y_conv,1), feed_dict={x: img1, y_: label1,keep_prob: 1}))

[[0.0000083  0.00000108 0.00001092 0.00075321 0.00000212 0.9979874
  0.00007099 0.         0.00116384 0.00000208]]
[5]
